In [5]:
import pandas as pd
import numpy as np
from lifelines import CoxTimeVaryingFitter

def investigate_lags(
    df: pd.DataFrame,
    customer_id_col: str,
    event_col: str,
    col_to_be_lagged: str,
    lag_count: int,
    list_of_other_covariates=None,
    week_col: str = "week",
    penalizer: float = 0.01,
    robust: bool = False,
    binning: bool = False,
    bin_size: int = 2
):
    """
    Create multi-week lag features for 'col_to_be_lagged' (e.g., calls) up to 'lag_count' weeks,
    optionally bin them (e.g. lag1-2, lag3-4, etc.), add any additional covariates,
    then fit a Cox time-varying model to see which lags/bins are most influential.

    Parameters
    ----------
    df : pd.DataFrame
        The dataset, assumed sorted by (customer_id_col, week_col).
        Each row is (customer_id, week, event, ...).
    customer_id_col : str
        The column name identifying each customer (e.g. 'hcp_id').
    event_col : str
        The binary column name indicating if the event happened in this row's time slice
        (1 = event occurred at end of this interval, 0 = no event).
    col_to_be_lagged : str
        The column to create lag features for (e.g. 'calls').
    lag_count : int
        How many weekly lags to generate (1..lag_count).
    list_of_other_covariates : list of str, optional
        Additional columns you'd like to include as covariates (besides the lags).
    week_col : str, default 'week'
        The column name representing time (e.g. weeks).
    penalizer : float, default 0.01
        A small L2 penalty factor to help with collinearity or singularities in the Cox model.
    robust : bool, default False
        If True, fit the Cox model with robust standard errors.
    binning : bool, default False
        If True, sum adjacent lags into bins of size 'bin_size'. e.g. lag1-2, lag3-4, etc.
    bin_size : int, default 2
        The size of each bin if binning is True.

    Returns
    -------
    model : CoxTimeVaryingFitter
        The fitted lifelines CoxTimeVaryingFitter model.
    df_for_cox : pd.DataFrame
        The final DataFrame used to fit the model (including lagged or binned columns).
    """

    # 1. Copy df to avoid mutating the original
    data = df.copy()

    # If the user did not pass any additional covariates, default to empty list
    if list_of_other_covariates is None:
        list_of_other_covariates = []

    # 2. Generate the individual lag columns for col_to_be_lagged
    for lag in range(1, lag_count + 1):
        lag_col_name = f"{col_to_be_lagged}_lag{lag}"
        # Shift within each customer to get the value from 'lag' weeks ago
        data[lag_col_name] = data.groupby(customer_id_col)[col_to_be_lagged].shift(lag).fillna(0)

    # 3. If binning is True, sum up those lag columns in bins
    if binning:
        binned_columns = []
        step = bin_size
        # e.g., if lag_count=5 and bin_size=2 => we do bins: (1..2), (3..4), (5..5)
        for start_lag in range(1, lag_count+1, step):
            end_lag = min(start_lag + step - 1, lag_count)
            # Create a new binned column name
            if start_lag == end_lag:
                # Single-lag bin, e.g. 'lag5'
                binned_col = f"{col_to_be_lagged}_lag{start_lag}"
            else:
                # Multi-lag bin, e.g. 'lag1_2'
                binned_col = f"{col_to_be_lagged}_lag{start_lag}_{end_lag}"

            # Sum across the relevant lag columns
            lag_cols_to_sum = [
                f"{col_to_be_lagged}_lag{i}" for i in range(start_lag, end_lag + 1)
            ]
            data[binned_col] = data[lag_cols_to_sum].sum(axis=1)
            binned_columns.append(binned_col)

        # Remove the original un-binned lag columns
        # if you only want the binned versions
        for lag in range(1, lag_count+1):
            col_to_drop = f"{col_to_be_lagged}_lag{lag}"
            if col_to_drop in data.columns:
                data.drop(columns=[col_to_drop], inplace=True)

        # Final covariates = these new binned columns + user’s other covariates
        final_covariates = binned_columns + list_of_other_covariates
    else:
        # If binning=False, just keep the original individual lag columns
        final_covariates = [
            f"{col_to_be_lagged}_lag{i}" for i in range(1, lag_count+1)
        ] + list_of_other_covariates

    # 4. Create time interval columns for CoxTimeVarying
    data["start_time"] = data[week_col] - 1
    data["stop_time"] = data[week_col]

    # Build the final DataFrame for the Cox model
    df_for_cox = data[
        [customer_id_col, "start_time", "stop_time", event_col] + final_covariates
    ].copy()

    # 5. Fit Cox time-varying model
    ctv = CoxTimeVaryingFitter(penalizer=penalizer)
    ctv.fit(
        df_for_cox,
        id_col=customer_id_col,
        event_col=event_col,
        start_col="start_time",
        stop_col="stop_time",
        robust=robust
    )

    # 6. Print summary
    print("\n=== investigate_lags() Model Summary ===")
    print(f"Lagged variable: {col_to_be_lagged}, lag_count={lag_count}, binning={binning}, bin_size={bin_size}")
    ctv.print_summary()

    return ctv, df_for_cox


###############################################
# EXAMPLE USAGE (comment out if not needed)
###############################################
if __name__ == "__main__":
    # Example with a small dataset
    np.random.seed(123)
    df_example = pd.DataFrame({
        "cust_id": np.repeat([f"CUST_{i+1:02d}" for i in range(3)], 6),
        "week": list(range(1,7)) * 3,
    }).sort_values(["cust_id","week"]).reset_index(drop=True)

    # Suppose we have 'calls' and a binary 'event'
    df_example["calls"] = np.random.poisson(lam=2, size=len(df_example))
    # event is random for demonstration
    df_example["event"] = np.random.binomial(1, 0.2, size=len(df_example))
    df_example["some_other_var"] = np.random.randn(len(df_example))

    # Without binning
    print("---- NO BINNING (e.g. lag_count=3) ----")
    model_no_bin, df_no_bin = investigate_lags(
        df=df_example,
        customer_id_col="cust_id",
        event_col="event",
        col_to_be_lagged="calls",
        lag_count=3,
        list_of_other_covariates=["some_other_var"],
        week_col="week",
        binning=False,
        bin_size=2  # not used if binning=False
    )

    # With binning
    print("\n---- BINNING (lag_count=5, bin_size=2) ----")
    # we'll artificially set lag_count=5 just to show how leftover works
    model_bin, df_bin = investigate_lags(
        df=df_example,
        customer_id_col="cust_id",
        event_col="event",
        col_to_be_lagged="calls",
        lag_count=2,
        list_of_other_covariates=["some_other_var"],
        week_col="week",
        binning=False,
        bin_size=2
    )


---- NO BINNING (e.g. lag_count=3) ----

=== investigate_lags() Model Summary ===
Lagged variable: calls, lag_count=3, binning=False, bin_size=2


<lifelines.CoxTimeVaryingFitter: fitted with 18 periods, 3 subjects, 3 events>
         event col = 'event'
         penalizer = 0.01
number of subjects = 3
 number of periods = 18
  number of events = 3
partial log-likelihood = -1.44
  time fit was run = 2025-01-14 02:48:08 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
calls_lag1      -0.08       0.92       1.51            -3.05             2.88                 0.05                17.90
calls_lag2       0.22       1.24       0.69            -1.14             1.57                 0.32                 4.83
calls_lag3      -0.16       0.85       0.88            -1.88             1.56                 0.15                 4.77
some_other_var   1.83       6.26       1.79            -1.68             5.35                 0.19               211.05

                 cmp to     z    p   -log2(p)
covariate                                    
calls_lag1         0.00 -0.05 0.96       0.06
calls_lag2         0.00  0.31 0.75       0.41
calls_lag3         0.00 -0.18 0.86       0.22
some_other_var     0.00  1.02 0.31       1.70
---
Partial AIC = 10.88
log-likelihood ratio test = 2.90 on 4 df
-log2(p) of ll-ratio test = 0.80


---- BINNING (lag_count=5, bin_size=2) ----

=== investigate_lags() Model Summary ===
Lagged variable: calls, lag_count=2, binning=False, bin_size=2


<lifelines.CoxTimeVaryingFitter: fitted with 18 periods, 3 subjects, 3 events>
         event col = 'event'
         penalizer = 0.01
number of subjects = 3
 number of periods = 18
  number of events = 3
partial log-likelihood = -1.46
  time fit was run = 2025-01-14 02:48:08 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
calls_lag1       0.04       1.04       1.35            -2.60             2.68                 0.07                14.54
calls_lag2       0.26       1.30       0.65            -1.01             1.54                 0.36                 4.66
some_other_var   1.96       7.11       1.60            -1.18             5.10                 0.31               164.03

                 cmp to    z    p   -log2(p)
covariate                                   
calls_lag1         0.00 0.03 0.98       0.04
calls_lag2         0.00 0.40 0.69       0.54
some_other_var     0.00 1.23 0.22       2.18
---
Partial AIC = 8.91
log-likelihood ratio test = 2.87 on 3 df
-log2(p) of ll-ratio test = 1.28

In [7]:
import pandas as pd
import numpy as np
from lifelines import CoxTimeVaryingFitter

def investigate_lags(
    df: pd.DataFrame,
    customer_id_col: str,
    event_col: str,
    col_to_be_lagged: str,
    lag_count: int,
    list_of_other_covariates=None,
    week_col: str = "week",
    penalizer: float = 0.01,
    robust: bool = False
):
    """
    Create multi-week lag features for 'col_to_be_lagged' (e.g., calls) up to 'lag_count' weeks,
    add optional covariates, then fit a Cox time-varying model to see which lags are most influential.

    Parameters
    ----------
    df : pd.DataFrame
        The dataset, assumed sorted by customer_id_col and then by week_col ascending.
        Each row is (customer_id, week, event, ...).
    customer_id_col : str
        The column name identifying each customer (e.g., 'hcp_id', 'customer_id').
    event_col : str
        The binary column name indicating if the event happened in this row's time slice.
        (1 = event occurred at end of this interval, 0 = no event).
    col_to_be_lagged : str
        The column to create lag features for (e.g., 'calls').
    lag_count : int
        How many weekly lags to generate (1..lag_count).
    list_of_other_covariates : list of str, optional
        Any additional columns you'd like to include as covariates (besides the lags).
    week_col : str, default 'week'
        The column name representing time (e.g., weeks).
    penalizer : float, default 0.01
        A small L2 penalty factor to help with collinearity or singular matrices.
    robust : bool, default False
        If True, fits the Cox model with robust standard errors.

    Returns
    -------
    model : CoxTimeVaryingFitter
        The fitted lifelines CoxTimeVaryingFitter model.
    df_for_cox : pd.DataFrame
        The final DataFrame used to fit the model (including lagged columns).
    """

    # 1. Copy df to avoid mutating the original
    data = df.copy()

    # Optional: verify it is sorted (if not sure it’s sorted, uncomment next line)
    # data = data.sort_values([customer_id_col, week_col]).reset_index(drop=True)

    if list_of_other_covariates is None:
        list_of_other_covariates = []

    # 2. Create lag columns
    # For each 'lag' in 1..lag_count, shift that many weeks *within each customer_id*
    for lag in range(1, lag_count + 1):
        new_col_name = f"{col_to_be_lagged}_lag{lag}"
        data[new_col_name] = data.groupby(customer_id_col)[col_to_be_lagged].shift(lag).fillna(0)

    # 3. Prepare data for time-varying Cox
    # We'll assume each row covers [week-1, week) in time.
    data["start_time"] = data[week_col] - 1
    data["stop_time"] = data[week_col]

    # Build list of final covariates: the newly created lag columns + the optional covars
    lagged_covars = [f"{col_to_be_lagged}_lag{i}" for i in range(1, lag_count + 1)]
    final_covars = lagged_covars + list_of_other_covariates

    # We'll keep only relevant columns
    df_for_cox = data[
        [customer_id_col, "start_time", "stop_time", event_col] + final_covars
    ].copy()

    # 4. Fit Cox time-varying model
    ctv = CoxTimeVaryingFitter(penalizer=penalizer)
    ctv.fit(
        df_for_cox,
        id_col=customer_id_col,
        event_col=event_col,
        start_col="start_time",
        stop_col="stop_time",
        robust=robust
    )

    # 5. Print summary to see hazard ratios for each lag
    print(f"\n=== Investigate Lags for {col_to_be_lagged} (up to {lag_count} weeks) ===")
    ctv.print_summary()

    return ctv, df_for_cox


###############################################
# EXAMPLE USAGE (comment out if not needed)
###############################################
if __name__ == "__main__":
    # Simulate a minimal example
    np.random.seed(123)
    df_example = pd.DataFrame({
        "cust_id": np.repeat([f"CUST_{i+1:02d}" for i in range(5)], 10),
        "week": list(range(1, 11)) * 5,
    }).sort_values(["cust_id","week"]).reset_index(drop=True)

    # Suppose we have 'calls', 'some_other_var', and a binary 'event'
    df_example["calls"] = np.random.poisson(lam=1.5, size=len(df_example))
    df_example["some_other_var"] = np.random.normal(loc=0, scale=1, size=len(df_example))

    # We'll define a trivial event process: if calls > 3, higher chance of event
    df_example["event"] = (df_example["calls"] > 3).astype(int)

    # Then let's run the function
    model, df_used = investigate_lags(
        df=df_example,
        customer_id_col="cust_id",
        event_col="event",
        col_to_be_lagged="calls",
        lag_count=2,
        list_of_other_covariates=["some_other_var"],
        week_col="week",
        penalizer=0.01,
        robust=False
    )

    # The model summary will show calls_lag1 and calls_lag2 coefficients,
    # plus 'some_other_var' if it's not dropped. 



=== Investigate Lags for calls (up to 2 weeks) ===


<lifelines.CoxTimeVaryingFitter: fitted with 50 periods, 5 subjects, 2 events>
         event col = 'event'
         penalizer = 0.01
number of subjects = 5
 number of periods = 50
  number of events = 2
partial log-likelihood = -1.65
  time fit was run = 2025-01-14 02:50:18 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
calls_lag1       1.08       2.93       1.00            -0.88             3.03                 0.41                20.70
calls_lag2      -0.97       0.38       0.81            -2.56             0.63                 0.08                 1.88
some_other_var   0.07       1.07       0.66            -1.23             1.37                 0.29                 3.94

                 cmp to     z    p   -log2(p)
covariate                                    
calls_lag1         0.00  1.08 0.28       1.83
calls_lag2         0.00 -1.19 0.24       2.09
some_other_var     0.00  0.10 0.92       0.13
---
Partial AIC = 9.30
log-likelihood ratio test = 3.14 on 3 df
-log2(p) of ll-ratio test = 1.43